# EXPORTS JOBS POSTED FROM [web.byui.edu/StudentEmployment/’](web.byui.edu/StudentEmployment/) INTO A CSV FILE

In [1]:
import http.client
import json
import pandas as pd
from bs4 import BeautifulSoup

## REQUEST DATA FROM API ENDPOINT

In [22]:
conn = http.client.HTTPSConnection("web.byui.edu")
 
conn.request("GET", "/studentemployment/api/jobs")

res = conn.getresponse()

data = res.read()

info = data.decode("utf-8")

responseObject = json.loads(info)

data_jobs = pd.DataFrame(responseObject)

## DISPLAY DATA RECEIVED

In [23]:
data_jobs

,jobID,title,departmentName,description,summary,displayJob,dateUpdated,startDate,endDate,managerName,...,beginningDate,recruitingStartDate,requireResume,limitApplicants,limitNumber,applicants,jobQuestions,isOnline,allowOnline,jobMajors
0,341246,BIO381 OnlineTutor,Campus Student,<p><b> <span>Online Tutor is needed for the BI...,Online Tutor is needed for the BIO 381: Pathop...,True,2022-08-27T19:43:28.8618258,2022-08-27T22:00:00,2022-09-13T06:00:00,Casey McDaniel,...,2022-09-13T06:00:00,2018-07-19T01:00:00,True,False,0,[],[],True,False,[]
1,1422088,BIO230 OnlineTutor,Online Student,<p><span> </span><span>Online Tutor is needed ...,Online Tutor is needed for the BIO 230: Essent...,True,2022-08-27T19:36:02.7437945,2022-08-27T08:00:00,2022-09-15T22:00:00,Casey McDaniel,...,2022-09-17T21:00:00,2020-11-13T01:00:00,True,False,0,[],[],False,True,[]
2,340938,BIO230 OnlineTutor,Campus Student,<p><b><span>Online</span><span> Tutor is neede...,Online Tutor is needed for the BIO 230: Essent...,True,2022-08-27T19:35:49.9147792,2022-08-27T20:00:00,2022-09-15T15:00:00,Casey McDaniel,...,2022-09-17T15:00:00,2018-08-17T01:00:00,True,False,0,[],[],True,False,[]
3,2181125,SMMBC160 OnlineTutor,Campus Student,<p>Online Tutor is needed for the SMMBC 160: S...,Online Tutor is needed for the SMMBC 160: Soci...,True,2022-08-27T19:02:04.8674657,2022-08-24T06:00:00,2022-09-08T06:00:00,Casey McDaniel,...,2022-09-12T06:00:00,2021-06-15T01:00:00,True,False,0,[],[],True,False,[]
4,2181124,SMMBC160 OnlineTutor,Online Student,<p>Online Tutor is needed for the SMMBC 160: S...,Online Tutor is needed for the SMMBC 160: Soci...,True,2022-08-27T19:02:04.8205847,2022-08-24T23:00:00,2022-09-08T18:00:00,Casey McDaniel,...,2022-09-12T23:00:00,2021-06-15T01:00:00,True,False,0,[],[],False,True,[]
5,1433782,SMMBC105 OnlineTutor,Campus Student,<p><span>Online Tutor is needed for the SMMBC ...,Online Tutor is needed for the SMMBC 105: Intr...,True,2022-08-27T19:01:51.9261804,2022-08-09T09:00:00,2022-09-08T03:00:00,Casey McDaniel,...,2022-09-12T06:00:00,2020-11-17T01:00:00,True,True,50,[],[],True,False,[]
6,1433781,SMMBC105 OnlineTutor,Online Student,<p><span> Online Tutor is needed for the SMMBC...,Online Tutor is needed for the SMMBC 105: Intr...,True,2022-08-27T19:01:51.8949303,2022-08-09T06:00:00,2022-09-08T03:00:00,Casey McDaniel,...,2022-09-12T06:00:00,2020-11-17T01:00:00,True,True,50,[],[],False,True,[]
7,1391534,SMMBC120 OnlineTutor,Campus Student,<p>Online Tutor is needed for the SMMBC 120: S...,Online Tutor is needed for the SMMBC 120: Soci...,True,2022-08-27T19:01:50.3305139,2022-08-19T17:00:00,2022-09-08T15:00:00,Casey McDaniel,...,2022-09-12T15:00:00,2020-11-06T01:00:00,True,True,50,[],[],True,False,[]
8,1391533,SMMBC120 OnlineTutor,Online Student,<p>Tutor is needed for the SMMBC 120: Social M...,Online Tutor is needed for the SMMBC 120: Soci...,True,2022-08-27T19:01:50.2837596,2022-08-19T11:00:00,2022-09-08T09:00:00,Casey McDaniel,...,2022-09-12T09:00:00,2020-11-06T01:00:00,True,True,50,[],[],False,True,[]
9,508139,TESOL101 OnlineTutor,Campus Student,<p>Online Tutor for TESOL 101: Introduction to...,Online Tutor for TESOL 101: Introduction to TE...,True,2022-08-27T19:01:25.9618198,2022-06-22T06:00:00,2022-09-07T06:00:00,Casey McDaniel,...,2022-09-12T06:00:00,2019-08-22T01:00:00,True,False,0,[],[],True,False,[]


## SIMPLE OVERVIEW

In [24]:
print(f'''
    THERE ARE {data_jobs.jobID.nunique()} JOBS.
    HIGHEST PAY JOB:\'{data_jobs[data_jobs.payRate==data_jobs.payRate.max()]['title'].iloc[0]}\' WITH {data_jobs.payRate.max()} DOLLARS AN HOUR.
    ONLINE JOBS: {data_jobs[data_jobs["title"].str.contains('Online')].shape[0]} OUT OF {data_jobs.shape[0]}.
''')


    THERE ARE 412 JOBS.
    HIGHEST PAY JOB:'Health Center Lab Assistant' WITH 11.5 DOLLARS AN HOUR.
    ONLINE JOBS: 360 OUT OF 412.



## CLEAN THE DATA

In [25]:
col_dates = ['dateUpdated','startDate','endDate','beginningDate','recruitingStartDate']

data_jobs[col_dates] = data_jobs[col_dates].astype('datetime64[ns]')

columns_to_drop = [
    'jobID', # not needed for EDA
    # 'description', # not needed for EDA
    # 'summary', # not needed for EDA
    'displayJob', # single boolean
    'dateUpdated','startDate','endDate', # not needed for display
    'approximateHoursPerWeek', # not consistent
    'positionsAllocated', # not relevant
    'positionsAvailble', # not relevant
    # 'workSchedule', # not needed for EDA
    'requireResume', # not relevant
    'limitApplicants', # not relevant
    'limitNumber', # not relevant
    'applicants', # empty
    'jobQuestions', # empty
    'isOnline', # not accurate
    'allowOnline', # not accurate
    'jobMajors' # not relevant
    ]

data_jobs.drop(columns_to_drop, axis=1, inplace=True)

data_jobs['description'] = data_jobs['description'].apply(lambda x: [p.text.strip() for p in BeautifulSoup(x).find_all('p') if p.text.strip() != ''])

## OPTIONAL: Save as a CSV

In [26]:
# data_jobs.to_csv('StudentEmployment.csv')

## FILTER OUT THE JOBS THAT HAVE 'Online','Custodian', and 'TA' IN THEIR TITLE

In [27]:
# JOBS FILTERED. NOT ONLINE, NOR CUSTODIAN, AND NOR TA.

remove = ['Online','Custodian','TA']

data_filtered = data_jobs[~data_jobs["title"].str.contains('|'.join(remove))].sort_values('payRate',ascending=False).reset_index().drop('index',axis=1)

print(f'{data_filtered.shape[0]} JOBS OUT OF {data_jobs.shape[0]} POSTED JOBS.')

pd.set_option('display.max_rows', None)

data_filtered

39 JOBS OUT OF 412 POSTED JOBS.


,title,departmentName,description,summary,managerName,payRate,workSchedule,beginningDate,recruitingStartDate
0,Health Center Lab Assistant,Student Health Center,"[PRINCIPLE DUTIES, REQUIREMENTS, APPLICATION I...","Assist lab manager in phlebotomy, performance ...",Emma Taylor,11.50,NaN,2022-09-12 12:00:00,2022-04-20 01:00:00
1,Clinician Aide,Student Health Center,"[PRINCIPLE DUTIES, REQUIREMENTS, APPLICATION I...",Assist health care providers and nursing staff...,Emma Taylor,10.15,M-F TBD,2022-09-01 12:00:00,2022-03-21 01:00:00
2,Food Services Student Jobs,University Food Services,"[Fast-paced, demanding job. We need dependabl...",Fall Food Service Jobs Start at $10 PER HOUR a...,Fauneil Schultz,10.00,Many Shifts Available,2022-09-16 00:00:00,2022-06-01 01:00:00
3,Data Entry - Alumni,Alumni,[Please read this posting thoroughly and caref...,This position enters data into the Alumni data...,Kathy Godfrey,10.00,Flexible - M-F 8-5 pm,2022-09-13 00:00:00,2022-08-22 01:00:00
4,Employer Relations Content Creator,Career & Employment Services,[The Employer Relations Content Creator will b...,Come join the growing and exciting BYU-Idaho C...,Janet Barton,10.00,Monday - Friday,2022-09-05 06:00:00,2022-07-26 01:00:00
5,Graphic Designer,Music,[Graphic Designer for the Department of Music....,Create posters and digital ads for department ...,Maria Nate,10.00,Flexible,2022-09-02 06:00:00,2018-07-23 01:00:00
6,Student Project Manager,Online Curriculum Development,[The student project manager position undertak...,Assist in managing the audiovisual section of ...,Brian Carter,10.00,Varies,2022-09-09 12:00:00,2016-11-28 01:00:00
7,Office Operations Student Lead,Housing and Student Living,"[Knowledge:, Apartment Living Standards, Stude...",The Office Operations Student Lead works direc...,Andrew Merrick,10.00,M-F: 1-5 pm,2022-09-01 18:00:00,2021-07-28 01:00:00
8,Systems & Innovation: Technology Specialist,Online Quality Assurance,"[Responsibilities, Trouble-shoot issues within...","As a Technology Specialist, you will be expect...",Corey Moore,9.60,"Monday-Saturday (7 AM to 10 PM, flexible shifts)",2022-09-10 06:00:00,2022-01-21 01:00:00
9,Systems & Innovation: Product Specialist,Online Quality Assurance,"[Responsibilities, Research educational techno...","As a Product Specialist, you will be expected ...",Corey Moore,9.60,"Monday-Saturday (7 AM to 10 PM, flexible shifts)",2022-09-10 20:00:00,2022-01-21 01:00:00


## OPTIONAL: Save as a CSV

In [8]:
# data_filtered.to_csv('StudentEmployment_filtered.csv')